In [ ]:
# Weather and Air Quality Data Integration
import os
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Function to fetch data from API
def fetch_data(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        if 'error' in data:
            raise ValueError(f"API error: {data['error']}")
        return data
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None

# Get API keys from environment variables
weather_api_key = os.getenv('OPENWEATHERMAP_API_KEY')
air_quality_api_key = os.getenv('AQICN_API_KEY')

if not weather_api_key or not air_quality_api_key:
    print("API keys are missing. Please set them as environment variables.")
    exit()

# Get city name from user
city = input("Enter the city name: ").strip()

# Fetch weather data
weather_url = f'http://api.openweathermap.org/data/2.5/weather?q={city}&appid={weather_api_key}'
weather_data = fetch_data(weather_url)

# Fetch air quality data
air_quality_url = f'http://api.waqi.info/feed/{city}/?token={air_quality_api_key}'
air_quality_data = fetch_data(air_quality_url)

# Process weather data
weather_info = {
    'Temperature (°C)': weather_data.get('main', {}).get('temp', 0) - 273.15,
    'Humidity (%)': weather_data.get('main', {}).get('humidity'),
    'Pressure (hPa)': weather_data.get('main', {}).get('pressure'),
    'Weather': weather_data.get('weather', [{}])[0].get('description')
}

# Process air quality data
air_quality_info = {
    'AQI': air_quality_data.get('data', {}).get('aqi'),
    'PM2.5 (µg/m³)': air_quality_data.get('data', {}).get('iaqi', {}).get('pm25', {}).get('v'),
    'PM10 (µg/m³)': air_quality_data.get('data', {}).get('iaqi', {}).get('pm10', {}).get('v'),
    'O3 (µg/m³)': air_quality_data.get('data', {}).get('iaqi', {}).get('o3', {}).get('v')
}

# Combine data into a DataFrame
combined_data = {**weather_info, **air_quality_info}
df = pd.DataFrame([combined_data])

# Display the DataFrame
print(df)

# Data Analysis
correlation_matrix = df.corr()

# Data Visualization - Correlation Heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Matrix of Weather and Air Quality Data')
plt.show()

# Data Visualization - Bar Chart
plt.figure(figsize=(10, 6))
df.plot(kind='bar', figsize=(10, 6))
plt.title('Weather and Air Quality Metrics')
plt.ylabel('Values')
plt.xlabel('Metrics')
plt.legend(loc='best')
plt.show()
